In [31]:
import numpy as np
import pandas as pd
import pickle

from collections import defaultdict

import sklearn.metrics

In [3]:
data_dir = '/s/project/mll/sergey/effect_prediction/MLM/'

In [141]:
utr_variants = pd.read_csv(data_dir + 'clinvar/clinvar.3utr.tsv', sep='\t', names=['chrom','pos','var_id','ref','alt','seq_name','label'])

In [142]:
utr_variants[utr_variants.seq_name.str.contains(',')]

,chrom,pos,var_id,ref,alt,seq_name,label
17,chr1,2405090,875342,G,A,"ENST00000605895.6_utr3_6_0_chr1_2403125_f,ENST...",Likely_benign
18,chr1,2405171,875344,C,T,"ENST00000605895.6_utr3_6_0_chr1_2403125_f,ENST...",Likely_benign
19,chr1,2405414,296265,C,T,"ENST00000605895.6_utr3_6_0_chr1_2403125_f,ENST...",Benign
239,chr1,21822298,876262,T,G,"ENST00000344642.7_utr3_4_0_chr1_21822216_f,ENS...",Likely_benign
240,chr1,21822324,295680,G,A,"ENST00000344642.7_utr3_4_0_chr1_21822216_f,ENS...",Benign
...,...,...,...,...,...,...,...
22205,chrX,19360517,914970,G,A,"ENST00000422285.7_utr3_10_0_chrX_19359654_f,EN...",Benign
22206,chrX,19360666,914971,C,A,"ENST00000422285.7_utr3_10_0_chrX_19359654_f,EN...",Benign
22207,chrX,19360712,914972,G,A,"ENST00000422285.7_utr3_10_0_chrX_19359654_f,EN...",Benign
22346,chrX,54446073,95076,G,A,"ENST00000375151.5_utr3_4_0_chrX_54444551_f,ENS...",Benign


In [105]:
utr_variants.label = utr_variants.label.apply(lambda x: 1 if 'athogenic' in x else 0)

In [106]:
utr_variants = utr_variants[~utr_variants.seq_name.str.contains(',')]

In [88]:
#embeddings for intact 3'UTR sequences

with open(data_dir + 'clinvar/species_aware_emb/all_3utr.pickle', 'rb') as f:
            embeddings = pickle.load(f)
            seqs =  pickle.load(f)
            
utr3_emb = {seq_name.split(':')[0]:embeddings[idx] for idx,seq_name in enumerate(seqs)}

In [93]:
#embeddings for alternative sequences

with open(data_dir + 'clinvar/species_aware_emb/clinvar.pickle', 'rb') as f:
            embeddings = pickle.load(f)
            seqs =  pickle.load(f)
            
clinvar_emb = {seq_name.split(':')[0]:embeddings[idx] for idx,seq_name in enumerate(seqs)}

In [137]:
from numpy.linalg import norm

def cosine(x,y):
    '''
    Compute cosine similarity btw 2 vectors
    '''
    if x is None or y is None:
        return None
    return  np.dot(x,y)/(norm(x)*norm(y))

In [138]:
utr_variants['y_pred'] = utr_variants.apply(lambda x: 
                                            cosine(utr3_emb.get(x.seq_name, None),
                                                   clinvar_emb.get(f'ClinvarID_{x.var_id}', None)),axis=1)

In [139]:
utr_variants = utr_variants[~utr_variants.y_pred.isna()]

In [140]:
sklearn.metrics.roc_auc_score(utr_variants.label, utr_variants.y_pred)

0.4791287429862864